In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import os
import sys  
sys.path.insert(0, r'C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data')
import warnings
from Signals_Processing import *
from Process import *
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import ensemble, feature_selection
import pickle

In [22]:
os.chdir('C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\')
os.getcwd()

'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system'

In [23]:
"""Load Data from Folder"""

folder = os.getcwd() + "\\Training Models\\"
participant = "P1_testing"
path = folder + participant

In [24]:
EEG_baseline = pd.read_csv(r'C:\Users\Rodrigo\Desktop\PhD\loop-system\Training Models\P1\EEGbaseline.csv', sep=";", index_col=0)

In [25]:
df_baseline = pd.read_csv(r'C:\Users\Rodrigo\Desktop\PhD\loop-system\Training Models\P1\baseline.csv', sep=";", index_col=0)

In [26]:
imp = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P1\\imp.pkl", "rb"))
scaler = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P1\\scaler.pkl", "rb"))
rfe = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P1\\rfe.pkl", "rb"))
model = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P1\\model.pkl", "rb"))

In [27]:
os.chdir(path)
(
    users,
    EEG_epochs,
    EEG_filtered,
    Signals_epochs,
    Signals,
    EEG_dict,
    features_signals,
    features_EEG,
    features_epochs,
    features_epochs_EEG,
) = ({}, {}, {}, {}, {}, {}, {}, {}, {}, {})

for root, dirs, files in os.walk(path):
    for fname in files:
        if fname.endswith(".xdf"):
            users[fname] = Run_files(fname)

In [28]:
users

{'P1_testing.xdf': [{'info': defaultdict(list,
               {'name': ['PsychoPy Ratings'],
                'type': ['Ratings'],
                'channel_count': ['2'],
                'channel_format': ['string'],
                'source_id': ['Example'],
                'nominal_srate': ['0.000000000000000'],
                'version': ['1.100000000000000'],
                'created_at': ['70829.67177770000'],
                'uid': ['6fd532ff-688f-498a-942e-2e1845466eea'],
                'session_id': ['default'],
                'hostname': ['DESKTOP-8HVM7BE'],
                'v4address': [None],
                'v4data_port': ['16574'],
                'v4service_port': ['16574'],
                'v6address': [None],
                'v6data_port': ['16574'],
                'v6service_port': ['16574'],
                'desc': [None],
                'stream_id': 3,
                'effective_srate': 0}),
   'footer': {'info': defaultdict(list,
                {'first_timestamp'

In [29]:
Opensignals_fs = 100
EEG_fs = 250
resolution = 16
sensors = ["ECG", "EDA", "RESP"]

In [30]:
data = {}
for user in users.keys():
    data[user.split(".")[0]] = Load_Data(
        users[user],
        "OpenSignals",
        "openvibeSignal",
        "PsychoPy Markers",
        "PsychoPy Markers",
        sensors,
    )

In [31]:
data

{'P1_testing': {'Signals':                Time      ECG     EDA     RESP
  0      93085.334763 -0.01150  1.8097 -0.01410
  1      93085.344759 -0.00988  1.7464 -0.01447
  2      93085.354755 -0.00970  1.7464 -0.01904
  3      93085.364752 -0.00359  1.7490 -0.01172
  4      93085.374748 -0.00144  1.7559 -0.01950
  ...             ...      ...     ...      ...
  63227  93717.366145 -0.11105  1.8082  0.08698
  63228  93717.376141 -0.10332  1.7879  0.09229
  63229  93717.386138 -0.09667  1.8024  0.08331
  63230  93717.396134 -0.09110  1.8307  0.08235
  63231  93717.406130 -0.08589  1.7754  0.08478
  
  [63232 rows x 4 columns],
  'EEG':                 Time         EEG_1         EEG_2         EEG_3         EEG_4  \
  0       93085.066135  92455.101562  73021.218750  90205.070312  88275.921875   
  1       93085.070135  92445.945312  73015.843750  90195.390625  88270.273438   
  2       93085.074135  92444.867188  73014.492188  90193.773438  88269.726562   
  3       93085.078135  92448.640

In [32]:
onset,offset = ({}, {})
for user in data.keys():
    for timestamp, markers in zip(data[user]["Markers Timestamps"], data[user]["Markers"]):
        if "start" in markers:
            onset[user] = timestamp
        elif "end" in markers:
            offset[user] = timestamp

In [33]:
onset, offset

({'P1_testing': 93105.03859090494}, {'P1_testing': 93714.74722529227})

In [34]:
onset_index = {}
offset_index = {}
onset_index_EEG = {}
offset_index_EEG = {}

for user in data.keys():
    onset_index[user] = np.where(data[user]["Signals"]["Time"] >= onset[user])[0]
    offset_index[user] = np.where(data[user]["Signals"]["Time"] >= onset[user])[0][-1]
    onset_index_EEG[user]  = np.where(data[user]["EEG"]["Time"] >= onset[user])[0]
    offset_index_EEG[user] = np.where(data[user]["EEG"]["Time"] >= onset[user])[0][-1]

In [35]:
len(onset_index['P1_testing'])

61260

In [36]:
Signals_epochs = {}
for users in data.keys():
    Signals_epochs[users] = nk.epochs_create(
        data[users]["Signals"],
        events=onset_index[users],
        sampling_rate=Opensignals_fs,
        epochs_start=0,
        epochs_end=40
    )
len(Signals_epochs['P1_testing'].keys())

61260

In [ ]:
EEG_filtered = filterEEG(data, EEG_fs)
EEG_filtered = getEEGChannels(EEG_filtered)
EEG_epochs = {}
for users in data.keys():
    EEG_epochs[users] = nk.epochs_create(
        data[users]["EEG"],
        events=onset_index[users],
        sampling_rate=EEG_fs,
        epochs_start=0,
        epochs_end=40
    )
len(EEG_epochs['P1_testing'].keys())

In [ ]:
features_EEG = getEEGBands(EEG_epochs, EEG_fs)
features_epochs_EEG, features_baseline_EEG = getEEGDict(features_EEG)